# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/mamba")

Mounted at /content/gdrive


In [ ]:
%pip install triton causal-conv1d

In [ ]:
%pip install --no-build-isolation --verbose .

In [4]:
import mamba_ssm

In [ ]:
!rm -rf build/

In [5]:
import torch, transformers, importlib, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

# Lab

In [31]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer("This version includes more back-and-forth dialogue, additional suggestions, and deeper emotional encouragement to stretch the duration while keeping it conversational. Let me know if it needs further tweaks!", return_tensors='pt').to('cuda')
model = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
tokenizer.batch_decode(model(**inputs).logits.argmax(-1))

[" is of the than-end-forth with, and\n, and more discussion and. the the boundaries of you the shortational.\n's know if you works a tweaking.\n"]

In [32]:
importlib.reload(InitMamba)
import InitMamba

model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
# model1.train()

res1 = model1(**inputs, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states
print(ssm_last_states[-1])

tensor([[[ 0.0114, -0.0793,  0.0059,  ...,  0.0306, -0.0761, -0.0956],
         [-0.0331,  0.0524, -0.0028,  ..., -0.0207,  0.0455,  0.0646],
         [-0.0869,  0.0827, -0.0114,  ..., -0.0583,  0.1434,  0.1890],
         ...,
         [ 0.0471, -0.0248,  0.0085,  ...,  0.0289, -0.0381, -0.0533],
         [-0.2410,  0.0452, -0.0167,  ..., -0.0806,  0.1820,  0.2515],
         [-0.0041,  0.0059, -0.0021,  ...,  0.0218,  0.0230,  0.0343]]],
       device='cuda:0', grad_fn=<SelectiveScanFnBackward>)


In [33]:
res = model1(**inputs, output_ssm_last_states = True, inputs_ssm_states = ssm_last_states)
tokenizer.batch_decode(res.logits.argmax(-1))

[' is includes more back-and-forth dialogue, additional suggestions, and deeper emotional encouragement to stretch the duration while keeping it conversational. Let me know if it needs further tweaks.\n']

In [34]:
res.logits

tensor([[[-47.9155, -61.0770, -45.7469,  ..., -61.2621, -60.9718, -61.3247],
         [-36.8091, -48.3119, -36.9276,  ..., -48.4115, -48.1929, -48.3928],
         [-54.3604, -67.9167, -53.6648,  ..., -67.8394, -67.7083, -68.0303],
         ...,
         [ 31.6040,   9.1483,  26.4352,  ...,   9.0277,   8.6580,   8.8144],
         [ 73.5115,  54.8170,  74.6049,  ...,  55.1642,  54.8182,  54.9174],
         [ -1.1340, -22.0350,  -6.8053,  ..., -22.0530, -22.1445, -22.0893]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)

In [35]:
ssm_last_states[0].mean().backward()

In [ ]:
# Model without initial SSM states gives random results
print(tokenizer.batch_decode(res1.logits.argmax(dim=-1)))

# Model with initial SSM states gives "Sunday"
res2 = model1(input_ids, inputs_ssm_states = ssm_last_states)
print(tokenizer.batch_decode(res2.logits.argmax(dim=-1)))

[', the,\n is Monday']
[', the,\n is Monday']


# Check same

In [ ]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config)

model1.eval()
model2.eval()

print((model2(input_ids).logits - model1(input_ids).logits).abs().sum())